#**Identifying Sleeper Builds in the NBA**

##**Objective**

A sleeper build is defined as someone who doesn't "look" the part but has tremendous skill that goes on display when they perform in their field of expertise. In the context of the NBA, a sleeper build is a player who visually does not look daunting, but when they check in on the court and play their game, they become a formidable opponent.
NBA teams are always in search of the next diamond in the rough...a player they can procure through a trade or offseason sign-on for a bargain, and will make a net positive impact. Players that come to mind historically are Steph Curry, Mugsy Bogues, and Nikola Jokic.
In this project, my goal is to first conduct EDA using Python and SQL (via Google BigQuery). I hope to uncover insights and KPIs that indicate which players are a sleeper build and could be a steal in the offseason or at the trade deadline. From there, I will create a Tableau dashboard to visualize my findings.

##**EDA (Python)**

In [1]:
# import all necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math
import os
import sys
import time
import datetime
import re
import json
import csv
import requests
import urllib
import io
import zipfile
import tarfile
import gzip
import shutil
import pickle
import hashlib
import base64
import collections
import itertools
import functools
import operator
import statistics
import scipy
import sklearn
import tensorflow as tf
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
nba_sleepers = pd.read_csv('/content/drive/MyDrive/Personal Projects/all_seasons.csv')

In [4]:
country_codes = pd.read_csv('/content/drive/MyDrive/Personal Projects/country_code.csv')

In [5]:
nba_sleepers.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97


In [6]:
nba_sleepers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12844 entries, 0 to 12843
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         12844 non-null  int64  
 1   player_name        12844 non-null  object 
 2   team_abbreviation  12844 non-null  object 
 3   age                12844 non-null  float64
 4   player_height      12844 non-null  float64
 5   player_weight      12844 non-null  float64
 6   college            10990 non-null  object 
 7   country            12844 non-null  object 
 8   draft_year         12844 non-null  object 
 9   draft_round        12844 non-null  object 
 10  draft_number       12844 non-null  object 
 11  gp                 12844 non-null  int64  
 12  pts                12844 non-null  float64
 13  reb                12844 non-null  float64
 14  ast                12844 non-null  float64
 15  net_rating         12844 non-null  float64
 16  oreb_pct           128

In [7]:
nba_sleepers.describe()

,Unnamed: 0,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct
count,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000,12844.000000
mean,6421.500000,27.045313,200.555097,100.263279,51.154158,8.212582,3.558486,1.824681,-2.226339,0.054073,0.140646,0.184641,0.513138,0.131595
std,3707.887763,4.339211,9.111090,12.426628,25.084904,6.016573,2.477885,1.800840,12.665124,0.043335,0.062513,0.053545,0.101724,0.094172
min,0.000000,18.000000,160.020000,60.327736,1.000000,0.000000,0.000000,0.000000,-250.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3210.750000,24.000000,193.040000,90.718400,31.000000,3.600000,1.800000,0.600000,-6.400000,0.021000,0.096000,0.149000,0.482000,0.066000
50%,6421.500000,26.000000,200.660000,99.790240,57.000000,6.700000,3.000000,1.200000,-1.300000,0.040000,0.130500,0.181000,0.525000,0.103000
75%,9632.250000,30.000000,208.280000,108.862080,73.000000,11.500000,4.700000,2.400000,3.200000,0.083000,0.179000,0.217000,0.563000,0.179000
max,12843.000000,44.000000,231.140000,163.293120,85.000000,36.100000,16.300000,11.700000,300.000000,1.000000,1.000000,1.000000,1.500000,1.000000


In [8]:
nba_sleepers = nba_sleepers.drop(['Unnamed: 0'], axis=1)

In [9]:
nba_sleepers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12844 entries, 0 to 12843
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player_name        12844 non-null  object 
 1   team_abbreviation  12844 non-null  object 
 2   age                12844 non-null  float64
 3   player_height      12844 non-null  float64
 4   player_weight      12844 non-null  float64
 5   college            10990 non-null  object 
 6   country            12844 non-null  object 
 7   draft_year         12844 non-null  object 
 8   draft_round        12844 non-null  object 
 9   draft_number       12844 non-null  object 
 10  gp                 12844 non-null  int64  
 11  pts                12844 non-null  float64
 12  reb                12844 non-null  float64
 13  ast                12844 non-null  float64
 14  net_rating         12844 non-null  float64
 15  oreb_pct           12844 non-null  float64
 16  dreb_pct           128

##**EDA (SQL)**

In [10]:
# download google bigquery api to use SQL
!pip install google-cloud-bigquery
from google.cloud import bigquery

In [11]:
# download google bigquery api to use SQL
!pip install google-cloud-bigquery
from google.cloud import bigquery

# **Authenticate with Google Cloud**
from google.colab import auth
auth.authenticate_user()

# Construct a BigQuery client object.
client = bigquery.Client()

# Set dataset_id to the ID of the dataset to fetch.
# The dataset_id should be in the format 'project_id.dataset_id'
project_id = 'ba775-shivnag'
dataset_id = 'nba_sleeper_builds'
dataset_ref = f"{project_id}.{dataset_id}" # Combine project_id and dataset_id

# Construct a full Dataset object to send to the API.
dataset = client.get_dataset(dataset_ref)  # Make an API request.

# View dataset properties
print(f"Got dataset '{dataset.dataset_id}'.")

Got dataset 'nba_sleeper_builds'.


In [12]:
# Assuming 'ba775-shivnag' is your project ID
client = bigquery.Client(project='ba775-shivnag') # Specify the project id for BigQuery Client

query_job = client.query("SELECT 1")
results = query_job.result()
print(list(results))

[Row((1,), {'f0_': 0})]


#**Defining True Shooting Percentage**

**Definition**: True shooting percentage (ts_pct in this dataset) measures a player's shooting efficiency. It considers all types of shots, including field goals, three-point shots, and free throws.

**Formula**: TS = PTS / 2*TSA, where:
- PTS = total points scored
- TSA: The total true shooting attempts, calculated as FGA + 0.44*FTA (Why 0.44? Through data analysis, researchers have determined that approximately 44% of all free throws are considered to take up a full possession)   
    - FGA: The total field goals attempted
    - FTA: The total free throws attempted

**Significance**: TS% is a more accurate way to measure a player's shooting ability than field goal percentage, three-point field goal percentage, or free throw percentage. It's a key metric for evaluating players and teams.

**Why TS%?**: Basketball is a game of points, and those with the ability to score can create a significant net positive impact for their team. Other variables are important such as AST, REB, etc. and those will be observed below and in Tableau, compared against TS%.

In [14]:
%%bigquery --project ba775-shivnag
SELECT player_name, ts_pct AS efficient_shooters
FROM ba775-shivnag.nba_sleeper_builds.nba_data
WHERE ts_pct > (0.513*2)
ORDER BY ts_pct DESC

Query is running:   0%|          |

Downloading:   0%|          |

,player_name,efficient_shooters
0,Anthony Brown,1.500
1,Jordan Sibert,1.500
2,Naz Mitrou-Long,1.500
3,Patrick McCaw,1.078
4,Tyrus Thomas,1.064
5,Dajuan Wagner,1.064
6,Donnell Harvey,1.064
7,Jerome James,1.064
8,Tyson Wheeler,1.064
9,Ryan Bowen,1.064


Only 13 players with TS% 2 Standard deviations (top 95%) in historical NBA (1994 to 2022)

In [16]:
%%bigquery --project ba775-shivnag
SELECT player_name, ts_pct, draft_number AS player_draft_num
FROM ba775-shivnag.nba_sleeper_builds.nba_data
WHERE ts_pct > (0.513*2)
ORDER BY draft_number ASC

Query is running:   0%|          |

Downloading:   0%|          |

,player_name,ts_pct,player_draft_num
0,Donnell Harvey,1.064,22
1,Anthony Brown,1.500,34
2,Jerome James,1.064,36
3,Patrick McCaw,1.078,38
4,Tyrus Thomas,1.064,4
5,Tyson Wheeler,1.064,47
6,Ryan Bowen,1.064,55
7,Dajuan Wagner,1.064,6
8,Tom Chambers,1.042,8
9,Braxton Key,1.064,Undrafted


Five of the top 13 were undrafted!